In [1]:
f = open('/media/sf_Windows/station_id.txt', 'r')

In [2]:
CD = []
STATION = []
ICAO = []
IATA = []
SYNOP = []

In [3]:
line = f.readline().strip()
country = str.split(line)[0]
line = f.readline().strip()

In [4]:
count = 1
while True:
    line = f.readline().strip()
    CD.append(line[0:2])
    STATION.append(line[3:20].strip())
    ICAO.append(line[20:24])
    IATA.append(line[26:29])
    SYNOP.append(line[32:37])
    if line == '':
        # either end of file or just a blank line.....
        # we'll assume EOF, because we don't have a choice with the while loop!
        line = f.readline().strip()
        country = str.split(line)[0]
        line = f.readline().strip()
        count += 1
        if count == 51:
            break
    

In [35]:
import pandas as pd
id_data = pd.DataFrame({'CD':CD,'STATION':STATION,'ICAO':ICAO,'IATA':IATA, 'ID': SYNOP})

In [36]:
airportHubs = ['ATL','LAX','ORD','DFW','JFK','DEN','SFO','CLT','LAS',
               'PHX','MIA','IAH','SEA','MCO','EWR','MSP','BOS','DTW',
               'PHL','LGA','FLL','BWI','DCA','MDW','SLC','IAD','SAN',
               'HNL','TPA','PDX','DAL','STL','HOU','AUS','BNA','OAK',
               'MSY','MCI','RDU','SNA','SJC','SMF','SJU','RSW','SAT',
               'CLE','PIT','IND','CMH','MKE','OGG','PBI','CVG','BDL',
               'JAX','ANC','BUF','ABQ','ONT','OMA']

In [37]:
ids = id_data[id_data.IATA.isin(airportHubs)].ID

In [38]:
ids[ids == '72243']

3092    72243
Name: ID, dtype: object

In [39]:
ids[460] = '747040'
ids[487] = '724839'
ids[498] = '724945'
ids[504] = '722977'
ids[686] = '747830'
ids[688] = '722108'
ids[722] = '722020'
ids[3089] = '722440'
ids[3556] = '726400'

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value 

In [40]:
id_data.loc[[498,504,686,688,722,3089,3556]]

,CD,IATA,ICAO,ID,STATION
498,CA,SJC,KSJC,74505,SAN JOSE
504,CA,SNA,KSNA,,COSTA MESA
686,FL,FLL,KFLL,,FT LAUDERD/HOLLY
688,FL,RSW,KRSW,,FORT MYERS
722,FL,MIA,KMIA,,MIAMI
3089,TX,HOU,KHOU,,HOUSTON/HOBBY
3556,WI,MKE,KMKE,,MILWAUKEE


In [41]:
newIds = []
for i in ids:
    if len(i) == 5:
        newIds.append(i + '0')
    else:
        newIds.append(i)

In [42]:
from ftplib import FTP
import zlib
ftp = FTP('ftp.ncdc.noaa.gov')
ftp.login()   

'230-****** WARNING ** WARNING ** WARNING ** WARNING ** WARNING ******\n ** This is a United States Department of Commerce computer     **\n ** system, which may be accessed and used only for             **\n ** official Government business by authorized personnel.       **\n ** Unauthorized access or use of this computer system may      **\n ** subject violators to criminal, civil, and/or administrative **\n ** action.  All information on this computer system may be     **\n ** intercepted, recorded, read, copied, and disclosed by and   **\n ** to authorized personnel for official purposes, including    **\n ** criminal investigations.  Access or use of this computer    **\n ** system by any person, whether authorized or unauthorized,   **\n ** constitutes consent to these terms.                         **\n ****** WARNING ** WARNING ** WARNING ** WARNING ** WARNING ******\n230 Anonymous access granted, restrictions apply'

In [43]:
def retrieveFileName(year):
    ftp.cwd("/pub/data/noaa/isd-lite/"+str(year))
    subfiles = ftp.nlst() 
    dirIDS = [str.split(file,'-')[0] for file in subfiles]
    desireFiles = [subfiles[dirIDS.index(val)] for val in newIds]
    return(desireFiles)

In [44]:
def get_gz(ftp, ftp_filename, local_filename):

    decomp = zlib.decompressobj(16+zlib.MAX_WBITS)

    unzip = open (local_filename, 'wb')

    def next_packet(data):
        unzip.write(decomp.decompress(data))

    ftp.retrbinary('RETR ' + ftp_filename, next_packet)

    decompressed = decomp.flush()
    unzip.write(decompressed)
    unzip.close()


In [45]:
fList = retrieveFileName(2012)

In [48]:
for i in range(len(fList)):
    get_gz(ftp, fList[i], 'weatherData/'+id_data[id_data.IATA.isin(airportHubs)].IATA.iloc[i]+'2012')